In [1]:
from agents import *
from notebook import psource


'''
In this context what we examine is the taxi driver world.Here the taxi driver is supposed to be driving in pre determined
pattern {zig zag} , find a customer and drop him to the desired location.
There are some more variables such as fuel levels and cab occupancy.

The problem can be formulated as follows:

• Initial state: Any state can be designated as the initial state however i have assumed it to be [0,0] 
• Actions: In this simple environment, each state has just following  actions: turn Left or  Right if a bump has been detected, move forward , pickup and drop a customer and fill petrol.
• Goal test: This whether all the existing customers have been serviced given adequate fuel levels. 
• Path cost: Each step costs 5 units of fuel. The agent would die if it runs out of fuel. 

''' 

In [2]:
class TheDumbTaxiDriver(Agent):
    current_cust_drop_loc=[2,2]
    is_customer_in=False
    iteration=0
    location=[0,1]
    direction=Direction('right')
    fuel_level=30
    def drive(self,thing):
        if  isinstance(thing,Customer):
            print('The agent is driving')
            return True
        else:
            False
    def hasEnoughFuel(self):
        if (self.fuel_level==0):
            return False
        else:
            return True
        
    def consumeFuel(self):
        self.fuel_level-=5
        return True
    
    def fillup(self,thing):
        if isinstance(thing,Petrol) and (self.fuel_level<50):
        
            print('Petrol low!!The agent is filling up gas..fuel level:{}'.format(self.fuel_level))
            self.fuel_level+=30
            return True
        else:
            print('Petrol enough!!The agent decided not to fill up gas..fuel_level:{}'.format(self.fuel_level))
            return False
        
        return False
    
    ''''def movedown(self):
        self.location += 1''' 
    
    def turn(self,d):
        print('Inside turn..turning {}'.format(d))
        self.direction= self.direction+d
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        self.consumeFuel()
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
        
        
        


In [3]:
class Customer(Thing):
    def __init__(self,drop_location=[1,1]):
        self.drop_location=drop_location
    
class Petrol(Thing):
    pass
class Bump(Thing):
    pass

    

class City(GraphicEnvironment):
    clues={
               'customers':[],
               'petrol_pumps':[]
              }
    def percept(self, agent):
        things=self.list_things_at(agent.location)
        
        
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
            
        return things
    
    def lookForCustomers(self, agent):
        things=self.list_things_at(agent.location)
        x=self.width
        y=self.height
        for x in range(10):
            for y in range(3):                
                customers=self.list_things_at([x,y],tclass=Customer)
                
                if(len(customers)!= 0):
                    for customer in customers:
                        self.clues['customers'].append([x,y])
                        print('Found and loaded the customer in the list at loc: {0},{1}'.format(x,y))
                
        
        return True
    
    def findPath(self,agent,current_location,to_location):
        
        
        print('The current_location:{0} , to_location:{1} '.format(current_location,to_location))
        
        x1=current_location[0]
        y1=current_location[1]
        x2=to_location[0]
        y2=to_location[1]
        for x in range( abs(x1-x2) ):
            if   (x2-x1)>0:
                agent.location[0]+=1
                self.reveal()
                print('The current_location:{0} , to_location:{1} '.format(agent.location,to_location))
            elif (x2-x1)<0:
                agent.location[0]-=1
                self.reveal()
                print('The current_location:{0} , to_location:{1} '.format(agent.location,to_location))
            else:
                pass
                
        for y in range( abs(y1-y2) ):
            if   (y2-y1)>0:
                agent.location[1]+=1
                self.reveal()
                print('The current_location:{0} , to_location:{1} '.format(agent.location,to_location))
            elif (y2-y1)<0:
                agent.location[1]-=1
                self.reveal()
                print('The current_location:{0} , to_location:{1} '.format(agent.location,to_location))
            else:
                pass
        if agent.location==to_location:
            print('Huuray!! Found my customer or dropped customer!!')
            return True
    
    def execute_action(self, agent, action):
        print('The width :{0} and height :{1} of the matrix is'.format(self.width,self.height))
        print('Inside Execute Action for Iteration:{0}  and action:{1} with direction:{2} and fuel level:{3}'.format(agent.iteration,action,agent.direction.direction,agent.fuel_level) )
        agent.iteration += 1
        self.lookForCustomers(agent)
        
        cust_pickup_location=self.clues['customers'][0]
        things=self.list_things_at(cust_pickup_location,tclass=Customer)
        cust_drop_location=things[0].drop_location
        
        if( self.findPath(agent,agent.location,cust_pickup_location) ):
            self.delete_thing(things[0])
            if (self.findPath(agent,agent.location,cust_drop_location) ):
                self.clues['customers'].pop(0)                            
                print('The remaining list of customers with the driver:{} '.format(self.clues['customers'])  )

        return action
    
    
    def is_done(self):
     
        no_edibles = not any(isinstance(thing, Customer) or isinstance(thing, Petrol) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        if(no_edibles):
            print('No more work  today..Let"s rest')
        return no_edibles or dead_agents
    

In [4]:
def program(percepts):
    
    for percept in percepts:
        
        if isinstance(percept,Customer):
            
            return 'pickup'
        elif isinstance(percept,Petrol):
            return 'fillup'
        elif isinstance(percept,Bump):
            
            return 'turn'

    
    return 'moveforward'


        
    

In [7]:
city = City(10,3, color={'TheDumbTaxiDriver': (200,0,0), 'Customer': (0, 200, 200), 'Petrol': (230, 115, 40)}) # park width is set to 5, and height to 20
theDumbTaxiDriver = TheDumbTaxiDriver(program)
customer = Customer(drop_location=[2,1])
customer2 = Customer(drop_location=[7,2])
customer3 = Customer(drop_location=[3,2])
petrol = Petrol()
more_petrol=Petrol()
city.add_thing(theDumbTaxiDriver,[0,0])
city.add_thing(customer3,[3,1])
city.add_thing(customer2,[5,2])
city.add_thing(customer,[6,0])
#city.add_thing(petrol,[0,1])
#city.add_thing(more_petrol,[1,0])

city.run(20)



,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


The width :10 and height :3 of the matrix is
Inside Execute Action for Iteration:0  and action:moveforward with direction:right and fuel level:30
Found and loaded the customer in the list at loc: 3,1
Found and loaded the customer in the list at loc: 5,2
Found and loaded the customer in the list at loc: 6,0
The current_location:[0, 0] , to_location:[6, 0] 


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


The current_location:[1, 0] , to_location:[6, 0] 


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


...

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


The current_location:[7, 2] , to_location:[7, 2] 
Huuray!! Found my customer or dropped customer!!
The remaining list of customers with the driver:[[6, 0], [3, 1], [6, 0], [3, 1], [3, 1], [5, 2], [6, 0], [3, 1], [5, 2], [5, 2]] 


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


No more work  today..Let"s rest


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
